# Profiling, Cython, and Numba 🚀
### Zbyszek & Jakob
### ASPP 2022, Bilbao, Spain

## Outline

* Introduction
* Profiling
* Speed up Python code using Cython
 * Basic principles
 * Interacting with NumPy arrays
* Using Numba to speed up Python code

## Introduction

* By now you are the *Master of Research*(tm).
![Master of research](figures/mor.png)
* Using your newly gained skills you can confidently transform any idea into a great manuscript.

* It seems like the only thing that's holding you back is the **execution speed** of your scripts!
* Both Cython and Numba are tools to make your code faster -> **optimization**.

## Exercise

Who thinks that they would benefit from reduced execution time?

Please raise your hand.

## The three rules of optimization
(adapted from Sebastian Witowski, EuroPython 2016)

#### 1. Don't.
 * Likely you don't need it.
 * Optimization comes with costs.

## Exercise

What are costs associated with optimization?

#### 2. Don't yet.
 * Is your code finished?
 * Did you write tests?
 * Are you sure it's worth the investment?

#### 3. Profile
* Don't guess which part of your code you should optimize!
* Measure. Measure. Measure.

## Runtime profilers

- profilers monitor the execution of your script and record, for example, how much time is spent in each function
- here we consider [py-spy](https://github.com/benfred/py-spy), a sampling-based profiler for Python
  - simply speaking `py-spy` examines your program at regular intervals and records which part is currently executed
- you can apply it to your script with `py-spy record -o profile.svg python myprogram.py`
  - to make timings accurate it needs to collect enough of data; you can control the "sampling rate" using the `-r` argument
- after measuring `py-spy` will produce a "flamegraph" like the following
![flamegraph](./figures/flamegraph.svg)

## Demo

Using a simple script, Jakob will explain how to read flamegraphs.

## Example: numerical integration

![RiemannSum](figures/MidRiemann2.svg)

Riemann sum: $\int_a^b dx f(x) \approx \sum_{i = 0}^{n - 1} f(a + (i + 0.5) \Delta x) \Delta x$ with $\Delta x = (b - a)/n$

here $a=0, b=2, n=4$

### Example implementation
(see [./profiling/numerical_integration.py](./profiling/numerical_integration.py))

Where do you think the bottlenecks are? *(don't do this at home!)*

## Demo

With your help, Jakob will demonstrate a typical profiling/optimization workflow based on this script.

## Exercise

It's time to put theory into practice. We have prepared an example script (see [./profiling/count_words.py](./profiling/count_words.py)) which counts the number of occurences of words in a text.

0. Fork & clone this repository.
1. Familarize yourself with the script.
2. Guess which functions are slow and should be optimized. *(don't do this at home.)*
3. Use the workflow (time -> py-spy- > timeit/lprun -> time) we have just demonstrated to reduce the script's execution time. **Make sure not to break the tests.**
4. Commit your changes in a new branch and create a PR. Include the duration before/after optimization in the PR message.

Afterwards we will discuss the exercise jointly.

## Exercise discussion

What did we learn?
- ...

## Profiling conclusion

- Before optimizing, first finish your code & write tests!
- Then *measure* to find slow functions. **Profiling is easy!**
- Only optimize the slowest functions & *know when to stop*!
- Most profilers can be invoked stand-alone and within ipython
  - `time` (commandline)
  - `%timeit`
  - `import timeit; timeit.time('some_func()')`
- [py-spy](https://github.com/benfred/py-spy) is just one of many profilers; alternatives:
  - [cProfile](https://docs.python.org/3/library/profile.html) + [snakeviz](https://github.com/jiffyclub/snakeviz)
  - [pyinstrument](https://github.com/joerick/pyinstrument)
- Here we focus on profiling *runtime*, but maybe you are limited by *memory*
  - [memray](https://github.com/bloomberg/memray)

### Optimization: what to do (in order of increasing complexity)

- Do nothing
- Vectorization (`numpy`!!)
- Data structures and algorithms
- Memoization / caching
- Non-Python libraries (`blas` vs. `openblas` vs. `atlas` vs. Intel `mkl`)
- Buy better hardware
- **Cython / Numba** / pythran
- **Parallelization** (->tomorrow)
- GPUs
- Low-level code


## Cython

In [37]:
# We start with our optimized Python implementation.
# Parameter 'f' has been renamed to 'func', to distinguish it from the 'f' function.

def f(x):
    return x**4 - 3 * x

def integrate_f(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [38]:
# We increase the number of points (from 5k to 1M) for easier measurement.

integrate_f(f, -10, +10, 1_000_000)

39999.999999868494

In [39]:
%timeit integrate_f(f, -10, +10, 1_000_000)

328 ms ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Our first step is to load the 'cython' extension and apply '%%cython' magic to the whole cell.

'f', 'integrate_f' are renamed to 'f2', 'integrate_f2'.

The code is otherwise unchanged.

In [4]:
%load_ext cython

In [40]:
%%cython

def f2(x):
    return x**4 - 3 * x

def integrate_f2(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [42]:
%timeit integrate_f2(f2, -10, +10, 1_000_000)

225 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# f2 and integrate_f2 are now compiled functions

f2, integrate_f2

(<function _cython_magic_9a59b8cb490d86577a5bc789129b6c05.f2>,
 <function _cython_magic_9a59b8cb490d86577a5bc789129b6c05.integrate_f2>)

Our next step is to set types of the function parameters.

This is called "type specialiation".

Our code is not valid Python anymore, only Cython can understand it.

In [8]:
%%cython

def f3(double x):
    return x**4 - 3 * x

def integrate_f3(func, double a, double b, int n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [9]:
integrate_f3(f3, -10, +10, 1_000_000)

39999.999999868494

In [10]:
%timeit integrate_f3(f3, -10, +10, 1_000_000)

157 ms ± 9.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Our next step is to also provide types of internal variables.

We also use '-a' to tell '%%cython' to generate annotated output with hints about Python ↔ C-code interation.

In [11]:
%%cython -a

def f4(double x):
    return x**4 - 3 * x

def integrate_f4(func, double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [12]:
integrate_f4(f4, -10, +10, 1_000_000)

39999.999999868494

In [13]:
%timeit integrate_f4(f4, -10, +10, 1_000_000)

92.2 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Exercise 01-cython-primes

Please open [01-cython-primes/exercise.ipynb](01-cython-primes/exercise.ipynb) and follow instructions therein.

### Cython function type specialization

We see strong yellow in the internal loop:

every time we call 'f', which is a Python function, conversion from Python types to c types and back must be done. 

This is slow.

In [14]:
%%cython

# We do "function specialization", by changing 'f5' to be a C function.
#
# On the first attempt we get a warning, because the generated C function is not used.

cdef double f5(double x):
    return x**4 - 3 * x

def integrate_f5(func, double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [15]:
%%cython

# In the second attempt, integrate_f5 hardcodes the function to call.
# We can rename it to just 'f', because it'll not be visible anywhere.

cdef double f(double x):
    return x**4 - 3 * x

def integrate_f5(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += f(a + (i + 0.5) * dx) * dx
    return s

In [16]:
%timeit integrate_f5(-10, +10, 1_000_000)

25.8 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Cython formula optimization

As a final optimization step, we can replace the call to `x**4`,
which generates a call to the C function `pow()`, with a series of multiplications.

We can also reorder the expression a bit to save one multiplication.

In [17]:
%%cython

cdef double f(double x):
    # This is the same as x**4 - 3*x, and also the same as x*x*x*x - 3*x
    return (x*x*x - 3) * x

def integrate_f6(double a, double b, int n):
    cdef:
        double dx = (b - a) / n
        double s = 0.0
        int i
    
    for i in range(n):
        s += f(a + (i + 0.5) * dx) * dx
    return s

In [18]:
integrate_f6(-10, +10, 1_000_000)

39999.999999868494

In [19]:
%timeit integrate_f6(10, +10, 1_000_000)

1.97 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Exercise: 02-cython-distrib

 Please open a terminal, `cd` to `02-cython-distrib/`, and follow the instructions:

In [20]:
from IPython import display
display.display(display.Markdown(open('02-cython-distrib/README.md').read()))


This is a very simple package that uses Cython.
It shows how to "package" a Python package with compiled sources.

The source code is just the `.pyx` file (`integrate_f6.pyx`).

The other files are "metadata":

- [pyproject.toml](02-cython-distrib/pyproject.toml)
- [meson.build](02-cython-distrib/meson.build)
- [setup.py](02-cython-distrib/setup.py)

Exercise:
    0) have a look at the different files
    1) install the module
    2) make sure it can be imported from anywhere:

    cd /
    python -c 'import integrate_f6; print(integrate_f6)'
    
    3) call the `integrate_f6` function from the Python interpreter after importing the module

--

How to install:

The "new" way is to use `pip`:

    pip install --user .

To just build a distributable bundle:

    pip wheel .

This creates a binary "wheel" (zip archive) in dist/ .

--

The "old" way is to use setup.py:

    python setup.py build

or

    python setup.py build_ext --inplace

to compile the code.

--

Optionally:

`meson` can also be invoked without `pip`:

    meson build-meson
    meson compile -C build-meson

This builds a module (underneath `build-meson/`), but does not put it the Python module search path.





### When `setup.py` and when `meson.build`?

[<img src="images/logo-over-white.svg" width="100"/>](images/logo-over-white.svg)

- setuptools is (still) the standard in the Python ecosystem
- excellent integration with PyPI and other Python packages
- automatic downloads from PyPI
- clumsy integration with non-Python libraries
- weak support for optional dependencies and partial rebuilds

[<img src="images/Meson_(software)_logo_2019.svg" width="180"/>](images/Meson_(software)_logo_2019.svg)

- Meson is arguably the best available build system for compiled code
- excellent integration with pkgconfig and other system libraries
- integration with PyPI via pip, somewhat clumsy
- excellent support for user configuration, optional dependencies, and partial rebuilds

Thus, if setuptools is a good solution for Python projects with some Cython code, and no dependencies on system libraries. Meson is a good solution for some self-contained Python and/or Cython code, possibly alongside other non-Python libraries and executables.

# Cython and Numpy Arrays

## Array summing

Let's start by summing up an array.

`const double [:]` means that the parameter is a immutable 1D memoryview.

We can use this function on numpy arrays.

In [21]:
%%cython -a

import numpy as np

def mysum(const double [:] A):
    cdef:
        double s = 0.0
        ssize_t i, n = A.shape[A.ndim - 1]
    
    for i in range(n):
        s += A[i]
    return s

In [22]:
x = np.linspace(0, 10, 54)
mysum(x)

270.00000000000006

In [23]:
%%cython -a

# From the yellow output above, we see that in line 11 a check
# is performed to optionally throw BufferIndexError.
# We know that this cannot happen, so we can tell cython to
# disable the check with 'boundscheck(False)'.
#
# While at it, let's disable support for negative indices with
# 'wraparound(False)'. We don't use those.

import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
def mysum(const double [:] A):
    cdef:
        double s = 0.0
        ssize_t i, n = A.shape[A.ndim - 1]
    
    for i in range(n):
        s += A[i]
    return s

## `mean3filter`

Let's write a "mean filter", i.e. a function that calculates a local
average for every 3 points.


$$ \{ x_0, x_1, ...  , x_{n-2}, x_n \} \longrightarrow \{ \frac{x_0 + x_1}{2}, \frac{x_0 + x_1 + x_2}{3}, \frac{x_1 + x_2 + x_3}{3}, ... , \frac{x_{i-1} + x_i + x_{i+1}}{3}, ... , \frac{x_{n-3} + x_{n-2} + x_{n-1}}{3}, \frac{x_{n-2} + x_{n-1}}{2} \} $$

In [24]:
import numpy as np

def mean3filter_p(arr):
    arr_out = np.empty_like(arr)
    
    arr_out[0] =  (arr[0] + arr[1]) / 2
    for i in range(1, arr.shape[0] - 1):
        arr_out[i] = arr[i-1:i+2].sum() / 3
    arr_out[-1] = (arr[-2] + arr[-1]) / 2

    return arr_out

## Exercise 03-cython-mean3filter

Please open [03-cython-mean3filter/exercise.ipynb](03-cython-mean3filter/exercise.ipynb) and follow instructions therein.

In [25]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [26]:
%%cython -a

# This is a naive cythonized version.

import numpy as np

def mean3filter_c1(const double [:] arr):
    cdef double [:] arr_out = np.empty_like(arr)
    cdef ssize_t i, N = arr.size
    
    arr_out[0] =  (arr[0] + arr[1]) / 2
    for i in range(1, N - 1):
        arr_out[i] = (arr[i-1] + arr[i] + arr[i+1]) / 3
    arr_out[-1] = (arr[-2] + arr[-1]) / 2

    return arr_out

In [27]:
%%cython -a

# Similarly to 'mysum', we don't need the bounds checks.
# But we are using negative indices. We can do get rid of
# this by changing '-1' to 'N-1' and '-2' to 'N-2'.
# With that change, we can also disable wraparound support.

import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
def mean3filter_c2(const double [:] arr):
    cdef double [:] arr_out = np.empty_like(arr)
    cdef ssize_t i, N = arr.size
    
    arr_out[0] =  (arr[0] + arr[1]) / 2
    for i in range(1, N - 1):
        arr_out[i] = (arr[i-1] + arr[i] + arr[i+1]) / 3
    arr_out[N-1] = (arr[N-2] + arr[N-1]) / 2

    return arr_out

In [28]:
x = np.random.rand(1_000_000)

In [29]:
%timeit mean3filter_p(x)

2.09 s ± 41.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit mean3filter_c1(x)

1.43 ms ± 47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%timeit mean3filter_c2(x)

1.48 ms ± 56.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


It turns out the cythonized versions are statistically indistinguishable.

Cython is about 750 times faster.

http://docs.cython.org/en/stable/src/userguide/memoryviews.html

# Wrapping external code in Cython

In [32]:
f3

<function _cython_magic_8333d4130183217cee6a930dacf6f59a.f3>

# Numba

In [44]:
import numba

# We apply the numba decorator to 'f' and 'integrate_f'
# to let numba genenerate optimized versions of those funtions.
#
# 'numba.njit' is the same as 'numba.jit(nopython=True)',
# and it means that numba should throw an exception instead of
# falling back to the python version, in cases where it cannot generate
# the optimized version.

@numba.njit
def f(x):
    return x**4 - 3 * x

@numba.njit
def integrate_f(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [45]:
%timeit integrate_f(f, -10, +10, 1_000_000)

2.11 ms ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
import numba

# Here we repeat the optimization of the formula to replace
# pow() by a multiplications. It turns out that this doesn't
# provide any speedup, so we can conclude that numba
# (or the compiler backend, llvm) were smart enough to do this
# step automatically.

@numba.njit
def f(x):
    return (x*x*x - 3) * x

@numba.njit
def integrate_f(func, a, b, n):
    dx = (b - a) / n
    s = 0.0
    for i in range(n):
        s += func(a + (i + 0.5) * dx) * dx
    return s

In [47]:
%timeit integrate_f(f, -10, +10, 1_000_000)

2.01 ms ± 62.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


[<img src="images/whiteboard.jpeg" width="200"/>](images/whiteboard.jpeg)

### Summary of Python vs. Cython vs. Numba

(Timings on my laptop, 64-bit amd-compatible cpu, 2.5 GHz)


```
original python version:      74   ms / 5000 iterations  = 14800 ns / iteration
with [].append                 3.7 ms / 5000 iterations  =   740 ns / iteration
no list                        3.3 ms / 5000 iterations  =   660
constant moved outside loop    3.0 ms / 5000 iterations  =   600
----
pure python:                  335    ms / 10⁶ iterations =   335 ns / iteration
python-compatible cython:     225    ms / 10⁶ iterations =   225 ns / iteration
typing of arguments:          157    ms / 10⁶ iterations =   157 ns / iteration
typing of variables:           92    ms / 10⁶ iterations =    92 ns / iteration
c-only function:               25    ms / 10⁶ iterations =    25 ns / iteration
simplified expression form:     2.11 ms / 10⁶ iterations =     2 ns / iteration

numba jit:                      2.01 ms / 10⁶ iterations =     2 ns / iteration
```

We made a speed-up of approximately 7400 times.

14800 ns × 2.5 × 10⁹ cycles/s = 37000 cycles
2 ns × 2.5 × 10⁹ cycles/s = 5 cycles

Our original versions was using about 40k CPU cycles (steps) to calculate a single iteration of the loop.
The final version requires 5 cycles. This is what we should expect for four multiplications,
a subtraction and an addition, and some overhead to manage the loop.

# Architecture of Cython and Numba

[<img src="images/cython_architecture.png" width="400"/>](images/cython_architecture.png)

[<img src="images/numba_architecture.png" width="400" />](images/numba_architecture.png)

 * ~Release the GIL and parallelize easily~ *(moved to parallel lecture)*
 * ~Wrap C/C++ code~ *(not relevant enough)*

https://cython.readthedocs.io/en/latest/src/userguide/parallelism.html